In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# U-Net Inspired Model
#def build_model(input_shape=(128, 128, 3)):
    #inputs = Input(input_shape)
    #c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    #p1 = MaxPooling2D((2, 2))(c1)
    #c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    #p2 = MaxPooling2D((2, 2))(c2)
    #c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    #u1 = UpSampling2D((2, 2))(c3)
    #c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
    #u2 = UpSampling2D((2, 2))(c4)
    #c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    #outputs = Conv2D(1, (1, 1), activation='linear', padding='same')(c5)
    #return Model(inputs, outputs)

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# RMSE Metric
def rmse(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))



from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

def build_model(input_shape=(128, 128, 3)):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze pretrained layers
    
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(512, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(128 * 128, activation='linear')(x)  # Predict depth directly
    outputs = tf.keras.layers.Reshape((128, 128, 1))(x)
    
    return Model(base_model.input, outputs)

# Data generator
def data_generator(rgb_dir, depth_dir, batch_size=32):
    files = sorted(os.listdir(rgb_dir))
    while True:
        for i in range(0, len(files), batch_size):
            batch_files = files[i:i+batch_size]
            imgs, depths = [], []
            for file in batch_files:
                img = cv2.imread(os.path.join(rgb_dir, file), cv2.IMREAD_COLOR)
                img = cv2.resize(img, (128, 128)) / 255.0
                depth = cv2.imread(os.path.join(depth_dir, file), cv2.IMREAD_UNCHANGED)
                depth = cv2.resize(depth, (128, 128)) / 255.0
                depth = np.expand_dims(depth, axis=-1)
                imgs.append(img)
                depths.append(depth)
            yield np.array(imgs), np.array(depths)

# Load test images
def load_test_images(test_dir):
    images, filenames = [], []
    for file in sorted(os.listdir(test_dir)):
        img = cv2.imread(os.path.join(test_dir, file), cv2.IMREAD_COLOR)
        img = cv2.resize(img, (128, 128)) / 255.0
        images.append(img)
        filenames.append(file)
    return np.array(images), filenames

# Save predictions as images
def save_predictions(predictions, filenames, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for pred, fname in zip(predictions, filenames):
        pred = pred.squeeze()  # Remove channel dimension
        pred_norm = (pred - np.min(pred)) / (np.max(pred) - np.min(pred) + 1e-6)
        pred_img = np.uint8(pred_norm * 255)
        cv2.imwrite(os.path.join(output_folder, fname), pred_img)

# Create simplified CSV in the format shown
def create_simplified_csv(predictions, filenames, output_csv):
    # Prepare data with just 125 dummy columns (based on the example)
    data = []
    
    for idx, (pred, fname) in enumerate(zip(predictions, filenames)):
        # Just create a row with zeros for all columns (as shown in the example)
        row = [idx, fname] + [0] * 125  # 125 zeros based on the sample
        data.append(row)
    
    # Create column names: id, ImageID, and numbers 0-124
    column_names = ["id", "ImageID"] + list(range(125))
    
    # Create DataFrame and save to CSV
    df = pd.DataFrame(data, columns=column_names)
    df.to_csv(output_csv, index=False)
    print(f"Simplified CSV created at {output_csv} with shape {df.shape}")

# Paths (adjust accordingly)
train_rgb, train_depth = '/kaggle/input/depth-estimation/competition-data/competition-data/training/images', '/kaggle/input/depth-estimation/competition-data/competition-data/training/depths'
val_rgb, val_depth = '/kaggle/input/depth-estimation/competition-data/competition-data/validation/images', '/kaggle/input/depth-estimation/competition-data/competition-data/validation/depths'
test_rgb = '/kaggle/input/depth-estimation/competition-data/competition-data/testing/images'
output_folder = '/kaggle/working/predictions'
output_csv = '/kaggle/working/predictions.csv'

from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

callbacks = [
    ModelCheckpoint('model.keras', monitor='val_rmse', save_best_only=True, mode='min'),
    EarlyStopping(monitor='val_rmse', patience=5, mode='min'),
    ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=3, mode='min')
]

import scipy.ndimage

def refine_depth(predictions):
    refined_preds = []
    for pred in predictions:
        pred = pred.squeeze()  # Remove extra dimension
        pred_smoothed = scipy.ndimage.gaussian_filter(pred, sigma=1)  # Apply Gaussian blur
        refined_preds.append(np.expand_dims(pred_smoothed, axis=-1))  # Restore channel dimension
    return np.array(refined_preds)

# Build & Compile Model
model = build_model()
model.compile(optimizer=Adam(1e-4), loss='mse', metrics=[rmse])

# Train Model
model.fit(
    data_generator(train_rgb, train_depth),
    epochs=50,
    steps_per_epoch=300,
    validation_data=data_generator(val_rgb, val_depth),
    validation_steps=50,
    callbacks=callbacks
)

# Load Best Model & Predict
model.load_weights('model.keras')
test_images, filenames = load_test_images(test_rgb)
predictions = model.predict(test_images)
predictions = refine_depth(predictions)  # Apply smoothing

# Save predictions & CSV
save_predictions(predictions, filenames, output_folder)
create_simplified_csv(predictions, filenames, output_csv)






In [ ]:
# Training
#model = build_model()
#model.compile(optimizer=Adam(), loss='mse', metrics=[rmse])
#model.fit(
    #data_generator(train_rgb, train_depth),
    #epochs=20,
    #steps_per_epoch=200,
    #validation_data=data_generator(val_rgb, val_depth),
    #validation_steps=30,
    #callbacks=[
        #ModelCheckpoint('model.keras', monitor='val_rmse', save_best_only=True, mode='min'),
        #EarlyStopping(monitor='val_rmse', patience=5, mode='min')
    #]
#)

In [ ]:
# Prediction
#model.load_weights('model.keras')
#test_images, filenames = load_test_images(test_rgb)
#predictions = model.predict(test_images)

# Save image predictions
#save_predictions(predictions, filenames, output_folder)

# Create the simplified CSV with the exact format shown
#create_simplified_csv(predictions, filenames, output_csv)

#print("Prediction and CSV generation complete.")